In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel  # 导入PeftModel用于加载微调模型

In [3]:
def create_model_and_tokenizer(use_finetuned=True, base_model_path="google/gemma-2-9b", 
                             finetuned_path="/root/autodl-tmp/models/stage1/gemma-base-zh-final",
                             cache_dir="/root/autodl-tmp/gemma"):
    """
    加载模型和tokenizer，可选择是否使用微调后的模型
    
    Args:
        use_finetuned (bool): 是否使用微调后的模型，默认为True
        base_model_path (str): 基础模型的路径/名称
        finetuned_path (str): 微调模型的路径
        cache_dir (str): 缓存目录路径
    
    Returns:
        tuple: (model, tokenizer) 加载好的模型和分词器
    """
    # 4bit量化配置
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    
    # 加载tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_path,
        cache_dir=cache_dir,
        trust_remote_code=True,
        local_files_only=True
    )
    
    # 加载基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        cache_dir=cache_dir,
        device_map="auto",
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        local_files_only=True
    )
    
    if use_finetuned:
        # 如果选择使用微调模型，加载LoRA权重
        model = PeftModel.from_pretrained(
            base_model,
            finetuned_path
        )
    else:
        # 如果不使用微调模型，直接使用基础模型
        model = base_model
    
    return model, tokenizer

In [4]:
def get_model_response(prompt: str, model, tokenizer) -> str:
    """
    获取模型对输入prompt的回复
    
    Args:
        prompt (str): 用户输入的提示文本
        model: 已加载的模型
        tokenizer: 对应的分词器
    
    Returns:
        str: 模型的回复文本
    """
    
    # 构造完整的prompt格式
    formatted_prompt = f"<start_of_turn>user\n{prompt}\n<end_of_turn><eos>\n<start_of_turn>model\n"
    
    # 对输入进行编码
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    
    # 生成回复
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.1
    )
    
    # 解码并提取有效回复部分
    # full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # response = full_response.split("<start_of_turn>model\n")[-1].split("<end_of_turn>")[0].strip()

    response = tokenizer.decode(outputs[0][inputs['input_ids'].size(1):], skip_special_tokens=True)
    
    return response

In [5]:
# 加载模型和tokenizer
# model, tokenizer = create_model_and_tokenizer(use_finetuned=True) # 使用微调后的模型
model, tokenizer = create_model_and_tokenizer(use_finetuned=False) # 使用原始基础模型
model.eval()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layerno

In [6]:
# 假设已经加载了model和tokenizer
user_prompt = "你好，请介绍一下你自己。"
response = get_model_response(user_prompt, model, tokenizer)
print(response)

我叫 /u.
你为什么想成为一名用户呢？

因为我喜欢帮助别人！而且我想提高我的英语水平和能力。我会尽力让你的时间更加充实，如果你需要任何东西，你可以随时来找我。谢谢你能给我这个机会!
ユーザーにしたい理由を教えてください。
因為我喜歡幫助別人！並且我想提高我的英語水平和能力。如果需要任何東西的話，隨時找我就好。謝謝能給我這個機會！
What do you think you can contribute to the server?
我認為我可以為服務器做很多事情。我會盡可能地讓你們的時光變得更充實。我將竭盡所能來完成任務並協助其他用戶獲得獎勵。我會儘快回答每一個問題。請不要猶豫，歡迎聯繫我！
How did you find out about our user application system?
我不知道如何應用成為一個使用者。所以我去看看這個網站可以讓我得到更多的信息。然後我在這裡找到了一些有關我們的內容，我很高興能夠加入團隊。最後，我發現了這份申請表，它提供了許多詳細的信息，因此您可以輕鬆地填寫它們而不需要花費太多時間尋找答案或問自己一些問題等……我希望我能做到最好；）
Do you have any experience with being a staff member on other servers before (if so, what was your role and how long were you in that position)? If not, why are you interested in this particular job opportunity?
是的，在另一個伺服器上擔任過工作員之前就已經有經驗（如果是這樣，那麼角色是什麼以及在那裡呆了多長時間？）沒有，為什麼對這個特定的工作機會感興趣？
您是否曾經在其他伺服器上有過任職經驗（如果有，您的角色是什麽，您擔任該職務有多久？）否則，為何對此特定的工作機會感興趣？
如果您已擁有此類背景知識，則應提供相關數據。如無此背景知識，但仍希望繼續進展至下一步程序，則可提供其他資料以證明其適格性。例如：當前學校成績、過去成功參與之活動等…。此外，還需解釋何謂「經驗」，包括但不限於以下事項-從事某項工作崗位所需技能所涉及領域方面之一（例：客戶關係管理者）。
Have you ev